In [ ]:
from dotenv import load_dotenv

load_dotenv()

### interrupt in compile step

In [ ]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage


class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool

In [ ]:
from langchain_core.tools import tool


@tool
def request_assistance():
    """전문가에게 대화를 에스컬레이션하세요. 직접 지원할 수 없거나 사용자가 귀하의 권한 외의 지원을 필요로 하는 경우 이 기능을 사용하세요.

    이 기능을 사용하려면 전문가가 적절한 안내를 제공할 수 있도록 사용자의 '요청'을 전달하세요.
    """
    return ""

@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["서울"]:
        return "기온은 섭씨 15도이고 흐립니다."
    else:
        return "기온은 섭씨 24도이고 화창합니다."

In [ ]:
tools = [get_weather]
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools + [request_assistance])


def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    ask_human = False
    if response.tool_calls and response.tool_calls[0]["name"] == "request_assistance":
        ask_human = True
    return {"messages": [response], "ask_human": ask_human}

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage


def create_response(response: str, ai_message: AIMessage):
    return ToolMessage(
        content=response,
        tool_call_id=ai_message.tool_calls[0]["id"],
    )


def human_node(state: State):
    new_messages = []
    if not isinstance(state["messages"][-1], ToolMessage):
        new_messages.append(
            create_response(
                "인기가 많은 레스토랑으로 한 달 전에 예약을 해야만 식사를 할 수 있습니다.",
                state["messages"][-1],
            )
        )
    return {
        "messages": new_messages,
        "ask_human": False,
    }


graph_builder.add_node("human", human_node)

In [ ]:
def select_next_node(state: State):
    if state["ask_human"]:
        return "human"
    return tools_condition(state)


graph_builder.add_conditional_edges(
    "chatbot",
    select_next_node,
    {"human": "human", "tools": "tools", "__end__": "__end__"},
)

In [ ]:
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")
graph_builder.set_entry_point("chatbot")
checkpointer = MemorySaver()
graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["human"],
)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "100"}}
input_message = HumanMessage(
    content="오늘 모수 레스토랑에 예약하고 싶은데, 당일 예약이 가능할까?"
)

graph.invoke({"messages": input_message}, config=config)

In [ ]:
graph.invoke(None, config=config)

### Human in the loop - Interrupt + Command Class

In [ ]:
from typing_extensions import Literal
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from IPython.display import Image, display


@tool
def weather_search(city: str):
    """Search for the weather in a specific city"""
    print("----")
    print(f"Searching for: {city}")
    print("----")
    return "Sunny!"


model = ChatOpenAI(model="gpt-4o-mini").bind_tools([weather_search])


def call_llm(state):
    return {"messages": [model.invoke(state["messages"])]}


def human_review_node(state) -> Command[Literal["call_llm", "run_tool"]]:
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[-1]

    human_review = interrupt(
        {
            "question": "해당 정보가 맞습니까?",
            "tool_call": tool_call,
        }
    )
    print("Human review values:", human_review)

    review_action = human_review["action"]
    review_data = human_review.get("data")

    if review_action == "continue":
        return Command(goto="run_tool")

    elif review_action == "update":
        updated_message = {
            "role": "ai",
            "content": last_message.content,
            "tool_calls": [
                {
                    "id": tool_call["id"],
                    "name": tool_call["name"],
                    "args": review_data,
                }
            ],
            "id": last_message.id,
        }
        return Command(goto="run_tool", update={"messages": [updated_message]})


def route_after_llm(state) -> Literal[END, "human_review_node"]:
    if len(state["messages"][-1].tool_calls) == 0:
        return END
    else:
        return "human_review_node"


builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)
builder.add_node("run_tool", ToolNode(tools=[weather_search]))
builder.add_node("human_review_node", human_review_node)
builder.add_conditional_edges("call_llm", route_after_llm)
builder.add_edge("run_tool", "call_llm")
builder.set_entry_point("call_llm")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
initial_input = {"messages": [HumanMessage(content="서욱의 날씨는 어때?")]}
config = {"configurable": {"thread_id": "5"}}

first_result = graph.invoke(initial_input, config=config, stream_mode="updates")
first_result

In [ ]:
print(graph.get_state(config).next)

In [ ]:
graph.invoke(Command(resume={"action": "continue"}), config=config)

In [ ]:
config = {"configurable": {"thread_id": "6"}}

graph.invoke(initial_input, config=config, stream_mode="updates")

In [ ]:
print(graph.get_state(config).next)

In [ ]:
graph.invoke(Command(resume={"action": "update", "data": {"city": "서울"}}), config)